MongoDB 데이터 삽입

In [1]:
from pymongo import MongoClient
from datetime import datetime

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": datetime(2023, 4, 12, 8, 0)},
        {"user_id": 1, "action": "view", "timestamp": datetime(2023, 4, 12, 9, 0)},
        {"user_id": 2, "action": "purchase", "timestamp": datetime(2023, 4, 12, 10, 0)},
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


Data inserted successfully


MongoDB 5문제 풀기  
**[문제 1: 특정 장르의 책 찾기]**  

- **문제 설명**:  
데이터베이스에 새로운 필드로 **`genre`**를 책 데이터에 추가하였습니다. 사용자는 "fantasy" 장르의 모든 책을 찾고 싶어합니다.  
- **쿼리 작성 목표**:  
"fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.  

In [8]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client.local

db.books.update_one(
    {"title": "Kafka on the Shore"},
    {"$set": {"genre": "fantasy"}}
)

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [9]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client.local

# fantasy 장르의 책 찾기
fantasy_books = db.books.find({ "genre": "fantasy" })

for book in fantasy_books:
    print(book)


{'_id': ObjectId('67e4d48ad464a45a14e2a9c7'), 'title': 'Kafka on the Shore', 'author': 'Haruki Murakami', 'year': 2002, 'genre': 'fantasy'}


**[문제 2: 감독별 평균 영화 평점 계산]**  

- **문제 설명**:  
각 영화 감독별로 그들의 영화 평점의 평균을 계산하고 싶습니다. 이를 통해 어떤 감독이 가장 높은 평균 평점을 가지고 있는지 알아볼 수 있습니다.  
- **쿼리 작성 목표**:  
모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.  

In [22]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client.local

pipeline = [
    {
        "$group": {
            "_id": "$director",
            "avgRating": { "$avg": "$rating" }
        }
    },
    {
        "$sort": { "avgRating": -1 }
    }
]

result = db.movies.aggregate(pipeline)
print("📊 감독별 평균 평점:")
for doc in result:
    print(f"{doc['_id']}: {round(doc['avgRating'], 2)}")

client.close()

📊 감독별 평균 평점:
Christopher Nolan: 8.8


**[문제 3: 특정 사용자의 최근 행동 조회]**  

- **문제 설명**:  
특정 사용자의 최근 행동 로그를 조회하고자 합니다. 이 때, 최신 순으로 정렬하여 최근 5개의 행동만을 보고 싶습니다.  
- **쿼리 작성 목표**:  
사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.  

In [26]:
from pymongo import MongoClient

def get_recent_actions(user_id):
    client = MongoClient("mongodb://localhost:27017/")
    db = client.local

    actions = db.user_actions.find(
        { "user_id": user_id }
    ).sort("timestamp", -1).limit(5)

    print(f"📄 사용자 {user_id}의 최근 행동 5개:")
    for action in actions:
        print(f"{action['timestamp']} - {action['action']}")

    client.close()

get_recent_actions(1)



📄 사용자 1의 최근 행동 5개:
2023-04-12 09:00:00 - view
2023-04-12 09:00:00 - view
2023-04-12 08:00:00 - click
2023-04-12 08:00:00 - click


**[문제 4: 출판 연도별 책의 수 계산]**

- **문제 설명** :
데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산하고자 합니다. 이 데이터는 시간에 따른 출판 트렌드를 분석하는 데 사용될 수 있습니다.
- **쿼리 작성 목표** :
각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [27]:
from pymongo import MongoClient

def count_books_by_year():
    client = MongoClient("mongodb://localhost:27017/")
    db = client.local

    pipeline = [
        {
            "$group": {
                "_id": "$year",
                "bookCount": { "$sum": 1 }
            }
        },
        {
            "$sort": { "bookCount": -1 }
        }
    ]

    results = db.books.aggregate(pipeline)

    print("📚 출판 연도별 책 수 (많은 순):")
    for doc in results:
        print(f"{doc['_id']}년: {doc['bookCount']}권")

    client.close()
count_books_by_year()

📚 출판 연도별 책 수 (많은 순):
1987년: 2권
2009년: 2권
2002년: 2권


**[문제 5: 특정 사용자의 행동 유형 업데이트]**  
- **문제 설명**:  
특정 사용자의 행동 로그 중, 특정 날짜 이전의 "view" 행동을 "seen"으로 변경하고 싶습니다. 예를 들어, 사용자 ID가 1인 사용자의 2023년 4월 13일 이전의 모든 "view" 행동을 "seen"으로 변경하는 작업입니다.  
- **쿼리 작성 목표**:  
사용자 ID가 1인 사용자의 2023년 4월 13일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결해 봅니다.  

In [28]:
from pymongo import MongoClient
from datetime import datetime

def update_view_to_seen(user_id, before_date):
    client = MongoClient("mongodb://localhost:27017/")
    db = client.local

    result = db.user_actions.update_many(
        {
            "user_id": user_id,
            "action": "view",
            "timestamp": { "$lt": before_date }
        },
        {
            "$set": { "action": "seen" }
        }
    )

    print(f"✅ 업데이트된 문서 수: {result.modified_count}개")

    client.close()
update_view_to_seen(1, datetime(2023, 4, 13))

✅ 업데이트된 문서 수: 2개
